In [3]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
from datasets import load_dataset
from tqdm import tqdm
import json
import torch
import argparse
import datasets
import numpy as np
import pandas as pd
# import matplotlib.pylab as plt
from os import listdir
import json
import math
# import seaborn as sns

In [4]:
from torch.nn.utils.prune import l1_unstructured
from torch.nn.utils import prune

In [5]:
cache_dir="/rds/general/user/jj1122/ephemeral/.cache/huggingface"
data_dir = "../m2d2/dataset"

In [6]:
def get_model_path(model_id):
    model_path_format = "/rds/general/user/jj1122/home/projects/m2d2/dataset/{model_id}/models"
    return model_path_format.format(model_id=model_id)
#     if model_id == "gpt2":
#         model_path = "gpt2"
# #         ckpt = "zs"
#     else:
#         model_path = model_path_format.format(model_id=model_id)
#         l_dir = listdir(model_path)

#         if all([len(x.split(".")) == 1 for x in l_dir]):
#             ckpt = max([int(x.split("-")[1]) for x in l_dir])
#             model_path += ckpt_path_format.format(ckpt=ckpt)
# #         else:
# #             ckpt = "final"
#     return model_path

In [7]:
# from, output model path 다르게 해서 해보자
# 전체 데이터 셋에 대해서 시행후 고고
# 모델, 데이터를 다르게 구분 지어보자

In [8]:
list_models = [
    "cs_l1",
    "nlin_l1",
    "Health_and_fitness",
    "Natural_and_physical_sciences",
    "Religion_and_belief_systems",
    "Culture_and_the_arts",
    "General_referece",
    "econ_l1",
    "History_and_events",
    "Human_activites",
    "Mathematics_and_logic",
    "astro-ph_l1",
    "cond-mat_l1",
#     "eess_l1 (No Metric)",
    "math_l1",
#     "physics_l1 (ERROR)",
    "q-bio_l1",
#     "q-fin_l1 (No Metric)",
    "stat_l1",
    "Philosophy",
    "Philosophy_and_thinking",
    "Society_and_social_sciences",
    "Technology_and_applied_sciences",
#     "art_l1 (No Data)"
]
len(list_models), len(set(list_models))

(20, 20)

In [9]:
device_id = 'cuda'
n_layers = 12
list_modules = ['ln_1', 'attn', 'ln_2', 'mlp']
trace_module_id = "transformer.h.{l}.{m}"

In [10]:
config_models = {model_id: {} for model_id in list_models}
for model_id in list_models:
    config_models[model_id]["model_path"] = get_model_path(model_id)
    json_path = config_models[model_id]["model_path"] + "/trainer_state.json"
    with open(json_path, "r") as json_file:
        trainer_state = json.load(json_file)
        config_models[model_id]["val_ppl"] = math.exp(trainer_state["best_metric"])
        config_models[model_id]["train_steps"] = int(trainer_state["best_model_checkpoint"].split("/")[-1].split("-")[-1])
        config_models[model_id]["train_eps"] = float(trainer_state["epoch"])

In [11]:
config_models['cs_l1']

{'model_path': '/rds/general/user/jj1122/home/projects/m2d2/dataset/cs_l1/models',
 'val_ppl': 9.60544432812854,
 'train_steps': 120000,
 'train_eps': 3.0348201502479255}

In [13]:
ds_nm = "Religion_and_belief_systems"

In [14]:
base_model = GPT2LMHeadModel.from_pretrained(config_models[ds_nm]["model_path"], cache_dir=cache_dir).to(device_id)


In [15]:
tuned_model = GPT2LMHeadModel.from_pretrained(config_models[ds_nm]["model_path"], cache_dir=cache_dir).to(device_id)

In [16]:
print(dict(tuned_model.named_buffers()).keys())  # to verify that all masks exist

dict_keys(['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias'])


In [17]:
tuned_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [18]:
list_pruning_modules_40_60 = ["h.2.attn", "h.3.attn", "h.4.mlp", "h.5.attn", "h.5.mlp", "h.0.mlp", "h.1.attn", "h.1.mlp", "h.2.mlp", "h.3.mlp"]
list_pruning_modules_0_20 = ["h.0.attn",  "h.9.mlp",  "h.10.mlp",  "h.11.mlp",  "h.8.attn",  "h.8.mlp",  "h.9.attn",  "h.10.attn",  "h.11.attn"]

In [19]:
for name, module in tuned_model.named_modules():
    for code in list_pruning_modules:
        
        if code in name and hasattr(module, "weight"):
            print (name)
            prune.ln_structured(module, name='weight', amount=0.5, n=2, dim=1)
#             prune.ln_structured(module, name='bias', amount=0.2, n=2, dim=0)
    
print(dict(tuned_model.named_buffers()).keys())  # to verify that all masks exist

transformer.h.0.mlp.c_fc
transformer.h.0.mlp.c_proj
transformer.h.1.attn.c_attn
transformer.h.1.attn.c_proj
transformer.h.1.mlp.c_fc
transformer.h.1.mlp.c_proj
transformer.h.2.attn.c_attn
transformer.h.2.attn.c_proj
transformer.h.2.mlp.c_fc
transformer.h.2.mlp.c_proj
transformer.h.3.attn.c_attn
transformer.h.3.attn.c_proj
transformer.h.3.mlp.c_fc
transformer.h.3.mlp.c_proj
transformer.h.4.mlp.c_fc
transformer.h.4.mlp.c_proj
transformer.h.5.attn.c_attn
transformer.h.5.attn.c_proj
transformer.h.5.mlp.c_fc
transformer.h.5.mlp.c_proj
dict_keys(['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.0.mlp.c_fc.weight_mask', 'transformer.h.0.mlp.c_proj.weight_mask', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.1.attn.c_attn.weight_mask', 'transformer.h.1.attn.c_proj.weight_mask', 'transformer.h.1.mlp.c_fc.weight_mask', 'transformer.h.1.mlp.c_proj.weight_mask', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transfo

In [20]:
tuned_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [21]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2", cache_dir=cache_dir)
dataset = load_dataset("machelreid/m2d2", ds_nm, cache_dir=cache_dir)
ds_test = dataset["test"].filter(lambda x: x['text'] != '')


Found cached dataset m2d2 (/rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00)
100%|██████████| 3/3 [00:00<00:00, 268.60it/s]
Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-b257c9c3236091fa.arrow


In [22]:
from transformers import set_seed

In [23]:
set_seed(718)

def tokenize_function(examples):
    output = gpt2_tokenizer(examples['text'], max_length=1024, truncation=True)
    return output

tokenized_datasets = ds_test.map(
    tokenize_function,
    batched=True,
    num_proc=8,
    remove_columns='text', #TODO
    load_from_cache_file=True,
)

len_sentences = len(tokenized_datasets)

Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-1629c4a1f2cfa6af_*_of_00008.arrow


In [24]:
list_results = [{} for x in range(len_sentences)]

In [25]:
from tqdm import tqdm

In [26]:
set_seed(718)

for sentence_idx, data in tqdm(enumerate(tokenized_datasets)):
#     if done_idx > sentence_idx: 
#         continue

    if sentence_idx % 1000 == 0:
        print(f"sentence_idx: {sentence_idx}")
#         with open(output_file, 'w') as json_file:
#             json.dump(list_results, json_file)

    inputs = torch.tensor(data['input_ids']).to(device_id)

    # First run: clean run
    clean_activations = {}
    with torch.no_grad():
        base_outputs = base_model(inputs, labels=inputs.clone())
        base_loss = np.exp(base_outputs.loss.item())

    # Second run: corrupted run
    with torch.no_grad():
        tuned_outputs = tuned_model(inputs, labels=inputs.clone())
        tuned_loss = np.exp(tuned_outputs.loss.item())


    list_results[sentence_idx]['base_loss'] = base_loss
    list_results[sentence_idx]['tuned_loss'] = tuned_loss

0it [00:00, ?it/s]

sentence_idx: 0


1009it [00:19, 58.97it/s]

sentence_idx: 1000


2008it [00:37, 58.66it/s]

...

87008it [25:28, 56.78it/s]

sentence_idx: 87000


88011it [25:46, 58.99it/s]

sentence_idx: 88000


88417it [25:53, 56.92it/s]


In [27]:
import pickle as pkl
f_nm = f"{ds_nm}_60_100.pkl"

with open(f_nm, "wb") as pkl_file:
    pkl.dump(list_results, pkl_file)

In [29]:
df_pd.DataFrame(list_results)

,base_loss,tuned_loss
0,63.918784,963.871829
1,33.487406,8596.633054
2,14.614459,9377.509797
3,27.317748,3508.981731
4,17.060393,5771.869686
5,30.848235,3702.990654
6,30.771209,3746.958354
7,1275.127155,14929.401446
8,62.453561,8972.282054
9,80.784695,5051.483341
